In [126]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


# Data description available at http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/adult-income-dataset/adult.csv


In [68]:
data = pd.read_csv('/kaggle/input/adult-income-dataset/adult.csv')

In [71]:
data.describe()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,1079.067626,87.502314,40.422382
std,13.710510,1.056040e+05,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.781445e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376420e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [70]:
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [72]:
def uniqueValues(data):
    for col in data:
        print(col,':\n',data[col].unique())
        print('---------------------------')

In [73]:
uniqueValues(data)

age :
 [25 38 28 44 18 34 29 63 24 55 65 36 26 58 48 43 20 37 40 72 45 22 23 54
 32 46 56 17 39 52 21 42 33 30 47 41 19 69 50 31 59 49 51 27 57 61 64 79
 73 53 77 80 62 35 68 66 75 60 67 71 70 90 81 74 78 82 83 85 76 84 89 88
 87 86]
---------------------------
workclass :
 ['Private' 'Local-gov' '?' 'Self-emp-not-inc' 'Federal-gov' 'State-gov'
 'Self-emp-inc' 'Without-pay' 'Never-worked']
---------------------------
fnlwgt :
 [226802  89814 336951 ... 129912 255835 257302]
---------------------------
education :
 ['11th' 'HS-grad' 'Assoc-acdm' 'Some-college' '10th' 'Prof-school'
 '7th-8th' 'Bachelors' 'Masters' 'Doctorate' '5th-6th' 'Assoc-voc' '9th'
 '12th' '1st-4th' 'Preschool']
---------------------------
educational-num :
 [ 7  9 12 10  6 15  4 13 14 16  3 11  5  8  2  1]
---------------------------
marital-status :
 ['Never-married' 'Married-civ-spouse' 'Widowed' 'Divorced' 'Separated'
 'Married-spouse-absent' 'Married-AF-spouse']
---------------------------
occupation :
 ['Machi

In [74]:
def missingValues(data):
    for c in data:
        if '?' in data[c].unique():
            print(c,':\n',data[c].value_counts()['?'])

In [75]:
missingValues(data)

workclass :
 2799
occupation :
 2809
native-country :
 857


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until


In [76]:
def removeMissing(data):
    st = []
    st.extend(data.loc[data['workclass']=='?'].index.tolist())
    st.extend(data.loc[data['native-country']=='?'].index.tolist())
    st.extend(data.loc[data['occupation']=='?'].index.tolist())
    stt = set(st)
    st = list(stt)
    data = data.drop(i for i in st)
    return data

In [77]:
data = removeMissing(data)

In [78]:
missingValues(data)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  This is separate from the ipykernel package so we can avoid doing imports until


In [83]:
#do the data pre-processing, select features.
def minMaxNorm(data,columns):
    minMax = preprocessing.MinMaxScaler()
    for c in columns:
        val = data[c].values
        data[c] = minMax.fit_transform(val.reshape(-1,1))
    return data

In [84]:
cols = data.select_dtypes([np.int64,np.float64]).columns
data = minMaxNorm(data,cols.tolist())
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.109589,Private,0.144430,11th,0.400000,Never-married,Machine-op-inspct,Own-child,Black,Male,0.000000,0.0,0.397959,United-States,<=50K
1,0.287671,Private,0.051677,HS-grad,0.533333,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.000000,0.0,0.500000,United-States,<=50K
2,0.150685,Local-gov,0.219011,Assoc-acdm,0.733333,Married-civ-spouse,Protective-serv,Husband,White,Male,0.000000,0.0,0.397959,United-States,>50K
3,0.369863,Private,0.099418,Some-college,0.600000,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,0.076881,0.0,0.397959,United-States,>50K
5,0.232877,Private,0.125398,10th,0.333333,Never-married,Other-service,Not-in-family,White,Male,0.000000,0.0,0.295918,United-States,<=50K


In [89]:
data['income'] = data['income'].replace('<=50K',1)

In [90]:
data['income'] = data['income'].replace('>50K',2)
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.109589,Private,0.144430,11th,0.400000,Never-married,Machine-op-inspct,Own-child,Black,Male,0.000000,0.0,0.397959,United-States,1
1,0.287671,Private,0.051677,HS-grad,0.533333,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.000000,0.0,0.500000,United-States,1
2,0.150685,Local-gov,0.219011,Assoc-acdm,0.733333,Married-civ-spouse,Protective-serv,Husband,White,Male,0.000000,0.0,0.397959,United-States,2
3,0.369863,Private,0.099418,Some-college,0.600000,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,0.076881,0.0,0.397959,United-States,2
5,0.232877,Private,0.125398,10th,0.333333,Never-married,Other-service,Not-in-family,White,Male,0.000000,0.0,0.295918,United-States,1


In [103]:
def oneHot(data,columns):
    data = pd.get_dummies(data, columns= columns)
    return data

In [104]:
cols = data.select_dtypes([object]).columns
data = oneHot(data, cols.tolist())
data.head()

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,income,workclass_Federal-gov,workclass_Local-gov,workclass_Private,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,0.109589,0.144430,0.400000,0.000000,0.0,0.397959,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,0.287671,0.051677,0.533333,0.000000,0.0,0.500000,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,0.150685,0.219011,0.733333,0.000000,0.0,0.397959,2,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,0.369863,0.099418,0.600000,0.076881,0.0,0.397959,2,0,0,1,...,0,0,0,0,0,0,0,1,0,0
5,0.232877,0.125398,0.333333,0.000000,0.0,0.295918,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0


,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.109589,Private,0.144430,11th,0.400000,Never-married,Machine-op-inspct,Own-child,Black,Male,0.000000,0.0,0.397959,United-States,1
1,0.287671,Private,0.051677,HS-grad,0.533333,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.000000,0.0,0.500000,United-States,1
2,0.150685,Local-gov,0.219011,Assoc-acdm,0.733333,Married-civ-spouse,Protective-serv,Husband,White,Male,0.000000,0.0,0.397959,United-States,2
3,0.369863,Private,0.099418,Some-college,0.600000,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,0.076881,0.0,0.397959,United-States,2
5,0.232877,Private,0.125398,10th,0.333333,Never-married,Other-service,Not-in-family,White,Male,0.000000,0.0,0.295918,United-States,1


In [105]:
train = data.sample(frac = 0.8, random_state=200)
test = data.drop(train.index)

In [106]:
print('Train Data: ', len(train), '\nTest Data: ',len(test))

Train Data:  36178 
Test Data:  9044


In [111]:
X_train = train.drop(['income'], axis = 1)
Y_train = train['income']

In [115]:
knn = KNeighborsClassifier(n_neighbors= 3)
knn.fit(X_train,Y_train)

KNeighborsClassifier(n_neighbors=3)

In [116]:
X_test = test.drop(['income'], axis = 1)
Y_test = test['income']

In [124]:
predictions = knn.predict(X_test)

In [127]:
print(accuracy_score(Y_test,predictions))

0.8123617868199912


In [128]:
print(confusion_matrix(Y_test,predictions))

[[6096  738]
 [ 959 1251]]


In [129]:
print(classification_report (Y_test,predictions))

              precision    recall  f1-score   support

           1       0.86      0.89      0.88      6834
           2       0.63      0.57      0.60      2210

    accuracy                           0.81      9044
   macro avg       0.75      0.73      0.74      9044
weighted avg       0.81      0.81      0.81      9044

